In [1]:
using InMemoryDatasets
import InMemoryDatasets as IMD
using BenchmarkTools
using TimerOutputs
using DLMReader
#using Revise
using Random

In [24]:
include("../src/main.jl")

cartesianjoin_timer (generic function with 1 method)

# Data

In [58]:
Random.seed!(792)
@show dsl = Dataset(xid = [111,222,333,444,222], 
                    x1 = [1,2,1,4,3], 
                    x2 = [-1.2,-3,2.1,-3.5,2.8],
                    #x3 = [Date("2019-10-03"), Date("2019-09-30"), Date("2019-10-04"), Date("2019-10-03"), Date("2019-10-03")],
                    x4 = ["abcd","efgh","ijkl","mnop","qrst"]);

@show dsr = Dataset(yid = [111,111,222,444,333],
                    y1 = [3,3,3,3,3],
                    y2 = [0,0, -3,1,2],
                    #y3 = [Date("2019-10-01"),Date("2019-10-01"), Date("2019-09-30"), Date("2019-10-05"), Date("2019-10-05")],
                    y4 = ["abc","abcd","efg","mnop","qrst"]);

dsl_big = Dataset(rand(10^3, 5), :auto);
modify!(dsl_big,[:x1,:x2,:x3] .=> byrow(x -> (x*100)));
modify!(dsl_big,[:x1,:x2,:x3] .=> byrow(round), [:x1,:x2,:x3] .=> byrow(Int));
modify!(dsl_big,[:x3] .=> byrow(string));

dsr_big = Dataset(rand(10^3, 5), :auto);
modify!(dsr_big,[:x1,:x2,:x3] .=> byrow(x -> (x*100)));
modify!(dsr_big,[:x1,:x2,:x3] .=> byrow(round), [:x1,:x2,:x3] .=> byrow(Int));
modify!(dsr_big,[:x3] .=> byrow(string));
function x2y(str)
  replace(str,'x' => 'y')
end
rename!(x2y,dsr_big);

dsl_big2 = Dataset(rand(10^4, 5), :auto);
modify!(dsl_big2,[:x1,:x2,:x3] .=> byrow(x -> (x*100)));
modify!(dsl_big2,[:x1,:x2,:x3] .=> byrow(round), [:x1,:x2,:x3] .=> byrow(Int));
modify!(dsl_big2,[:x3] .=> byrow(string));

dsr_big2 = Dataset(rand(10^4, 5), :auto);
modify!(dsr_big2,[:x1,:x2,:x3] .=> byrow(x -> (x*100)));
modify!(dsr_big2,[:x1,:x2,:x3] .=> byrow(round), [:x1,:x2,:x3] .=> byrow(Int));
modify!(dsr_big2,[:x3] .=> byrow(string));
function x2y(str)
  replace(str,'x' => 'y')
end
rename!(x2y,dsr_big2);

dsl = Dataset(xid = [111, 222, 333, 444, 222], x1 = [1, 2, 1, 4, 3], x2 = [-1.2, -3, 2.1, -3.5, 2.8], x4 = ["abcd", "efgh", "ijkl", "mnop", "qrst"]) = 5×4 Dataset
 Row │ xid       x1        x2        x4
     │ identity  identity  identity  identity
     │ Int64?    Int64?    Float64?  String?
─────┼────────────────────────────────────────
   1 │      111         1      -1.2  abcd
   2 │      222         2      -3.0  efgh
   3 │      333         1       2.1  ijkl
   4 │      444         4      -3.5  mnop
   5 │      222         3       2.8  qrst
dsr = Dataset(yid = [111, 111, 222, 444, 333], y1 = [3, 3, 3, 3, 3], y2 = [0, 0, -3, 1, 2], y4 = ["abc", "abcd", "efg", "mnop", "qrst"]) = 5×4 Dataset
 Row │ yid       y1        y2        y4
     │ identity  identity  identity  identity
     │ Int64?    Int64?    Int64?    String?
─────┼────────────────────────────────────────
   1 │      111         3         0  abc
   2 │      111         3         0  abcd
   3 │      222         3        -3  

In [44]:
filewriter("../files/dsl_big.csv", dsl_big)
filewriter("../files/dsr_big.csv", dsr_big)
filewriter("../files/dsl_big2.csv", dsl_big2)
filewriter("../files/dsr_big2.csv", dsr_big2)

# Time & allocations

## functions for join

In [66]:
function fun1(x,y) 
  x <= y
end

function fun2(x,y) 
  x >= y
end

function fun3(x,y) 
  length(x) == length(y)
end

fun3 (generic function with 1 method)

## small example

In [25]:
newds = cartesianjoin(dsl,dsr,on = [:xid=>:yid, :x1=>:y1=>fun1],
            multiple_match=[false,false],
            obs_id=true);
newds

5×9 Dataset
 Row │ xid       x1        x2        x4        y1        y2        y4        o ⋯
     │ identity  identity  identity  identity  identity  identity  identity  i ⋯
     │ Int64?    Int64?    Float64?  String?   Int64?    Int64?    String?   I ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │      111         1      -1.2  abcd             3         0  abc         ⋯
   2 │      111         1      -1.2  abcd             3         0  abcd
   3 │      222         2      -3.0  efgh             3        -3  efg
   4 │      333         1       2.1  ijkl             3         2  qrst
   5 │      222         3       2.8  qrst             3        -3  efg         ⋯
                                                               2 columns omitted

## big example
1000 rows * 1000 rows

In [59]:
newds = cartesianjoin_timer(dsl_big,dsr_big,
        on = [:x1=>:y1, :x2=>:y2=>fun1, :x3=>:y3=>isless]);

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:            146ms / 100.0%           5.97MiB / 100.0%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 compute flag vector        1    139ms   95.5%   139ms   5.73MiB   96.0%  5.73MiB
 generate new ds            1   6.56ms    4.5%  6.56ms    242KiB    4.0%   242KiB
   2-5 newds dsr            1   4.50ms    3.1%  4.50ms   97.0KiB    1.6%  97.0KiB
     5.2 fill _res          4   4.04ms    2.8%  1.01ms     64.0B    0.0%    16.0B
     5.1 init _res          4    127μs    0.1%  31.7μs   85.2KiB    1.4%  21.3KiB
   2-1 left row count       1   1.40ms    1.0%  1.40ms   12.0KiB    0.2%  12.0KiB
   2-2 left row

In [71]:
str = "76"
str[1]

'7': ASCII/Unicode U+0037 (category Nd: Number, decimal digit)

In [79]:
@time newds = cartesianjoin(dsl_big,dsr_big,
        on = [:x1=>:y1=>isless,:x2=>:y2=>isless, :x3=>:y3],check = false);
nrow(newds)

  0.010563 seconds (640 allocations: 1.191 MiB)


2351

## large example
10000 rows * 10000 rows

In [33]:
@time nds = cartesianjoin_timer(dsl_big2,dsr_big2,
              on = [:x1=>:y1, :x2=>:y2=>fun1, :x4=>:y4=>isless, :x4=>:y5=>fun2],
              check=false,obs_id=[true,true]);
nrow(nds)

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:            494ms / 100.0%           21.2MiB / 100.0%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 generate new ds            1    357ms   72.4%   357ms   21.2MiB   99.9%  21.2MiB
   2-5 newds dsr            1    253ms   51.3%   253ms   2.78MiB   13.1%  2.78MiB
     5.2 fill _res          4    253ms   51.2%  63.2ms     64.0B    0.0%    16.0B
     5.1 init _res          4    196μs    0.0%  48.9μs   2.76MiB   13.0%   708KiB
   2-1 left row count       1   84.5ms   17.1%  84.5ms    117KiB    0.5%   117KiB
   2-2 left row _res        1    780μs    0.2%   780μs   3.49MiB   16.5%  3.49MiB
     2.2 fill _

82777

In [104]:
@time nds = cartesianjoin(dsl_big2,dsr_big2,
              on = [:x1=>:y1=>isless,:x2=>:y2=>isless,:x3=>:y3,:x4=>:y4=>isless],
              check=false,obs_id=[true,true]);
nrow(nds)

  0.715783 seconds (9.20 k allocations: 120.490 MiB, 1.97% gc time)


119511

In [39]:
@time nds = cartesianjoin(dsl_big2,dsr_big2,on = [:x1=>:y1, :x2=>:y2, :x3=>:y3],
check=false);
nrow(nds)

  0.515975 seconds (519 allocations: 95.528 MiB, 6.09% gc time)


102

In [99]:
@time nds = innerjoin(dsl_big2,dsr_big2,on = [:x1=>:y1],check=false);
nrow(nds)

  0.044264 seconds (1.23 k allocations: 75.370 MiB, 28.88% gc time)


994920